In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("bank-additional-full.csv", sep=";")
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
df.shape

(41188, 21)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [5]:
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [7]:
df["subscribed"] = df["y"].map({"yes":1,"no":0})

In [9]:
df.replace("unknown",np.nan,inplace=True)

In [10]:
df.isna().sum()

age                  0
job                330
marital             80
education         1731
default           8597
housing            990
loan               990
contact              0
month                0
day_of_week          0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
emp.var.rate         0
cons.price.idx       0
cons.conf.idx        0
euribor3m            0
nr.employed          0
y                    0
subscribed           0
dtype: int64

In [11]:
df["contacted"] = 1

In [12]:
median_duration = df["duration"].median()

df["interested"] = np.where(
    (df["duration"] > median_duration) | (df["poutcome"] == "success"),
    1,
    0
)


In [13]:
contacted = df["contacted"].sum()
interested = df["interested"].sum()
subscribed = df["subscribed"].sum()

contact_to_interest = interested / contacted * 100
interest_to_subscribe = subscribed / interested * 100
overall_conversion = subscribed / contacted * 100

contact_to_interest, interest_to_subscribe, overall_conversion


(np.float64(50.6968048946295),
 np.float64(22.221157990517696),
 np.float64(11.265417111780131))

In [14]:
drop_contact_interest = 100 - contact_to_interest
drop_interest_subscribe = 100 - interest_to_subscribe

drop_contact_interest, drop_interest_subscribe



(np.float64(49.3031951053705), np.float64(77.7788420094823))

In [15]:
contact_funnel = df.groupby("contact").agg(
    contacted=("contacted", "sum"),
    interested=("interested", "sum"),
    subscribed=("subscribed", "sum")
)

contact_funnel["interest_rate_%"] = (
    contact_funnel["interested"] / contact_funnel["contacted"] * 100
)

contact_funnel["conversion_rate_%"] = (
    contact_funnel["subscribed"] / contact_funnel["interested"] * 100
)

contact_funnel


,contacted,interested,subscribed,interest_rate_%,conversion_rate_%
contact,,,,,
cellular,26144,13520,3853,51.713586,28.498521
telephone,15044,7361,787,48.929806,10.691482


In [16]:
job_funnel = df.groupby("job").agg(
    contacted=("contacted", "sum"),
    subscribed=("subscribed", "sum")
)

job_funnel["conversion_%"] = (
    job_funnel["subscribed"] / job_funnel["contacted"] * 100
)

job_funnel.sort_values("conversion_%", ascending=False)



,contacted,subscribed,conversion_%
job,,,
student,875,275,31.428571
retired,1720,434,25.232558
unemployed,1014,144,14.201183
admin.,10422,1352,12.972558
management,2924,328,11.217510
technician,6743,730,10.826042
self-employed,1421,149,10.485574
housemaid,1060,106,10.000000
entrepreneur,1456,124,8.516484


In [17]:
df.to_csv("bank_clean_funnel_data.csv", index=False)
contact_funnel.reset_index().to_csv("bank_contact_funnel.csv", index=False)
job_funnel.reset_index().to_csv("bank_job_funnel.csv", index=False)
